In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Sequential

In [2]:

tf.random.set_seed(42)
np.random.seed(42)


In [3]:

# 1) Load & preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = (x_train.astype("float32") / 255.0)  
x_test  = (x_test.astype("float32")  / 255.0) 

class_names = [
    "T-shirt/top","Trouser","Pullover","Dress","Coat",
    "Sandal","Shirt","Sneaker","Bag","Ankle boot"
]

In [4]:
x_train.shape

(60000, 28, 28)

In [18]:
model=Sequential([
    layers.Input(shape=(28, 28)),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
#     layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax"),
])


In [43]:
# Sequential
model=Sequential([
    layers.Input(shape=(28, 28)),
    layers.Flatten(),
    layers.Dense(50, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(50, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(50, activation="relu"),
    layers.Dense(10, activation="softmax"),
])


In [44]:
# functional
inputs = tf.keras.Input(shape=(28, 28), name="image")
x = layers.Flatten(name="flatten")(inputs)
x = layers.Dense(256, activation="relu", name="dense_1")(x)
x = layers.Dropout(0.3, name="dropout_1")(x)
x = layers.Dense(128, activation="relu", name="dense_2")(x)
x = layers.Dropout(0.2, name="dropout_2")(x)

outputs = layers.Dense(10, activation="softmax", name="predictions")(x)
# price = layers.Dense(1, activation="softmax", name="predictions")(x)

model = Model(inputs=inputs, outputs=outputs, name="fashion_mnist_mlp")
# modelp =  Model(inputs=inputs, outputs=price, name="fashion_mnist_mlp")

In [45]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [46]:
model.summary()

Model: "fashion_mnist_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               200960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
predictions (Dense)          (None, 10)          

In [47]:
# 3) Train (with EarlyStopping)
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True
)

history = model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=50,
    batch_size=128,
#     callbacks=[early_stop],
    verbose=2
)


Epoch 1/50
422/422 - 3s - loss: 0.6108 - accuracy: 0.7804 - val_loss: 0.4053 - val_accuracy: 0.8508
Epoch 2/50
422/422 - 2s - loss: 0.4308 - accuracy: 0.8448 - val_loss: 0.3610 - val_accuracy: 0.8645
Epoch 3/50
422/422 - 4s - loss: 0.3900 - accuracy: 0.8569 - val_loss: 0.3681 - val_accuracy: 0.8647
Epoch 4/50
422/422 - 4s - loss: 0.3654 - accuracy: 0.8661 - val_loss: 0.3483 - val_accuracy: 0.8717
Epoch 5/50
422/422 - 2s - loss: 0.3526 - accuracy: 0.8711 - val_loss: 0.3405 - val_accuracy: 0.8735
Epoch 6/50
422/422 - 2s - loss: 0.3371 - accuracy: 0.8756 - val_loss: 0.3412 - val_accuracy: 0.8805
Epoch 7/50
422/422 - 2s - loss: 0.3230 - accuracy: 0.8811 - val_loss: 0.3370 - val_accuracy: 0.8733
Epoch 8/50
422/422 - 2s - loss: 0.3152 - accuracy: 0.8831 - val_loss: 0.3203 - val_accuracy: 0.8820
Epoch 9/50
422/422 - 2s - loss: 0.3073 - accuracy: 0.8865 - val_loss: 0.3201 - val_accuracy: 0.8848
Epoch 10/50
422/422 - 2s - loss: 0.3000 - accuracy: 0.8892 - val_loss: 0.3145 - val_accuracy: 0.8925

In [48]:

# 4) Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

# 5) Quick predictions demo (first 5 test images)
probs = model.predict(x_test[:5])
preds = probs.argmax(axis=1)
print("\nSample predictions:")
for i, p in enumerate(preds):
    print(f"Image {i}: predicted={class_names[p]} (class {p}), true={class_names[y_test[i]]}")

# 6) (Optional) Save the model
# model.save("fashion_mnist_sequential_cnn.h5")



Test accuracy: 0.8966 | Test loss: 0.3263

Sample predictions:
Image 0: predicted=Ankle boot (class 9), true=Ankle boot
Image 1: predicted=Pullover (class 2), true=Pullover
Image 2: predicted=Trouser (class 1), true=Trouser
Image 3: predicted=Trouser (class 1), true=Trouser
Image 4: predicted=Shirt (class 6), true=Shirt


In [ ]:

# California Housing — Neural Network Regressor (Keras Functional API)
# ---------------------------------------------------------------
# Requirements: tensorflow>=2.9, scikit-learn, matplotlib (optional for plot)

import os
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

# ---------- Reproducibility ----------
# SEED = 42
# random.seed(SEED)
# np.random.seed(SEED)
# tf.keras.utils.set_random_seed(SEED)

# ---------- Load data ----------
# y is the median house value in units of $100,000 (original dataset definition)
X, y = fetch_california_housing(return_X_y=True)

# Train/val/test split: 64% train, 16% val, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=SEED
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.20, random_state=SEED
)

# ---------- Scale features ----------
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

n_features = X_train_s.shape[1]

# ---------- Build model (Functional API) ----------
def build_model(n_features: int) -> tf.keras.Model:
    inputs = tf.keras.Input(shape=(n_features,), name="features")
    x = tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.10)(x)

    x = tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal")(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(32, activation="relu", kernel_initializer="he_normal")(x)
    outputs = tf.keras.layers.Dense(1, name="median_house_value")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="california_housing_regressor")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss="mse",                                # optimize MSE
        metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_model(n_features)
model.summary()

# ---------- Training ----------
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=20,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
]

history = model.fit(
    X_train_s, y_train,
    validation_data=(X_val_s, y_val),
    epochs=200,
    batch_size=256,
    callbacks=callbacks,
    verbose=1
)

# ---------- Evaluate ----------
y_pred = model.predict(X_test_s).ravel()
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"\nTest RMSE: {rmse:.4f}")
print(f"Test MAE : {mae:.4f}")
print(f"Test R^2 : {r2:.4f}")

# ---------- Plot training curves ----------
plt.figure(figsize=(6,4))
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("MSE loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.tight_layout()
plt.show()

# ---------- (Optional) Save model & scaler ----------
# model.save("california_housing_regressor.keras")
# import joblib; joblib.dump(scaler, "scaler.joblib")



Model: "california_housing_regressor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
features (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               1152      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_28 (Dense)             (None, 32